In [ ]:
# import packages
import pandas as pd
import numpy as np
import os
import plotly.express as px

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

import download_data
import load_data
from column_checks import get_dtypes
from filepaths import *


year = 2021
path_prefix = f"{year}/"

## Load Consumed Grid Electricity Emission Factors from OGE

In [ ]:
# load hourly consumed emission rates

resolution = "hourly"
data_type = "carbon_accounting"

all_data = []
for ba in os.listdir(results_folder(f"{year}/{data_type}/{resolution}/us_units")):
    df = pd.read_csv(results_folder(f"{year}/{data_type}/{resolution}/us_units/{ba}"))
    df["ba_code"] = ba.split(".")[0]
    all_data.append(df)

all_data = pd.concat(all_data, axis=0)

# add columns for month and hour
all_data["month"] = (
    all_data["datetime_local"].str.split("-", expand=True)[1].astype(int)
)
all_data["hour"] = (
    all_data["datetime_local"]
    .str.split("-", expand=True)[2]
    .str.split(" ", expand=True)[1]
    .str.split(":", expand=True)[0]
    .astype(int)
)

## only keep columns that are relevant
all_data = all_data[
    [
        "ba_code",
        "datetime_utc",
        "datetime_local",
        "month",
        "hour",
        "consumed_co2_rate_lb_per_mwh_for_electricity",
    ]
]

all_data


## Load Vehicle Fuel Efficiency Data

In [ ]:
# download vehicle fuel economy data
model_year = 22

os.makedirs(downloads_folder("epa"), exist_ok=True)

url = f"https://www.fueleconomy.gov/feg/EPAGreenGuide/xls/all_alpha_{model_year}.xlsx"
filename = f"green_vehicle_guide_{model_year}.xlsx"
filepath = downloads_folder(f"epa/{filename}")
download_data.download_helper(url, filepath)

In [ ]:
# load the fuel economy data and calculate average mpge for all classes
vehicle_mpg = pd.read_excel(downloads_folder(f"epa/{filename}"))

vehicle_categories = {
    "car": ["large car", "small car", "midsize car", "station wagon"],
    "pickup": ["pickup"],
    "small_suv": ["small SUV"],
    "large_suv": ["standard SUV"],
}

avg_mpge_data = []
for fuel in ["Electricity", "Gasoline"]:
    for category in list(vehicle_categories.keys()):

        avg_mpge = (
            vehicle_mpg.loc[
                vehicle_mpg["Veh Class"].isin(vehicle_categories[category])
                & (vehicle_mpg["Fuel"] == fuel),
                "Cmb MPG",
            ]
            .astype("float32")
            .mean()
        )

        avg_mpge_data.append(
            pd.DataFrame(
                columns=["fuel_type", "vehicle_class", "mpge"],
                data=[[fuel.lower(), category, avg_mpge]],
            )
        )

avg_mpge_data = pd.concat(avg_mpge_data, axis=0)
avg_mpge_data


# Calculate Emissions per Mile Driven

In [ ]:
# constants
grid_gross_loss = 0.045 # per eGRID2021
gasoline_g_co2_per_gallon = 8780 # per EPA GHG Emissions Factor Hub
ethanol_g_co2_per_gallon = 5750 # per EPA GHG Emissions Factor Hub
co2_g_per_gallon = {"gasoline": gasoline_g_co2_per_gallon, "e10_gasoline": (gasoline_g_co2_per_gallon*0.9 + ethanol_g_co2_per_gallon*0.1)}
lb_per_g = 0.00220462


results = []

for fuel in ["gasoline", "e10_gasoline"]:
    for charging_loss in [0.10, 0.13, 0.2, 0.25]:
        for vehicle_class in ["car", "pickup", "large_suv", "small_suv"]:

            # calculate CO2 from gasmobile
            gasmobile_co2_lb_per_mi = (
                co2_g_per_gallon[fuel]
                * lb_per_g
                / avg_mpge_data.loc[
                    (avg_mpge_data["fuel_type"] == "gasoline")
                    & (avg_mpge_data["vehicle_class"] == vehicle_class),
                    "mpge",
                ].values[0]
            )

            # calculate kwh needed by EV
            ev_mi_per_kwh = (
                avg_mpge_data.loc[
                    (avg_mpge_data["fuel_type"] == "electricity")
                    & (avg_mpge_data["vehicle_class"] == vehicle_class),
                    "mpge",
                ].values[0]
                / 33.7
            )
            kwh_per_mi = 1 / ev_mi_per_kwh
            kwh_at_wall_per_mi = kwh_per_mi / (1 - charging_loss)
            kwh_generation_per_mi = kwh_at_wall_per_mi / (1 - grid_gross_loss)

            for charge_start_hour in [0, 9, 12, 18]:
                for charge_duration_hours in [1, 4, 8]:

                    calc = (
                        all_data[
                            (all_data["hour"] >= charge_start_hour)
                            & (
                                all_data["hour"]
                                < (charge_start_hour + charge_duration_hours)
                            )
                        ]
                        .groupby("ba_code")[
                            ["consumed_co2_rate_lb_per_mwh_for_electricity"]
                        ]
                        .mean()
                    )

                    calc["ev_co2_lb_per_mi"] = (
                        calc["consumed_co2_rate_lb_per_mwh_for_electricity"] / 1000
                    ) * kwh_generation_per_mi

                    calc["ev_pct_diff_gas"] = (
                        (calc["ev_co2_lb_per_mi"] - gasmobile_co2_lb_per_mi)
                        / gasmobile_co2_lb_per_mi
                    ).round(2)

                    calc["vehicle_class"] = vehicle_class
                    calc["charging_start"] = f"{charge_start_hour}:00"
                    calc["charge_duration"] = charge_duration_hours
                    calc["charging_loss"] = charging_loss
                    calc["gasoline_type"] = fuel

                    results.append(calc)

results = pd.concat(results, axis=0).reset_index()
results


In [ ]:
results[(results["ba_code"] == "SEC") & (results["vehicle_class"] == "large_suv")]

## Plot Results

In [ ]:
ba_name_dict = {'AEC': 'PowerSouth Energy Coop',
 'AECI': 'Associated Electric Coop',
 'AMPL': 'Anchorage Municipal Light & Power',
 'AVA': 'Avista Corporation',
 'AVRN': 'Avangrid Renewables',
 'AZPS': 'Arizona PSC',
 'BANC': 'BA of Northern California',
 'BPAT': 'Bonneville Power Admin.',
 'CEA': 'Chugach Electric Assn',
 'CHPD': 'PUD of Chelan County, WA',
 'CISO': 'California ISO',
 'CPLE': 'Duke Energy Progress East',
 'CPLW': 'Duke Energy Progress West',
 'CSTO': 'Constellation Energy',
 'DEAA': 'Arlington Valley',
 'DOPD': 'PUD #1 of Douglas Co., WA',
 'DUK': 'Duke Energy Carolinas',
 'EEI': 'Electric Energy',
 'EPE': 'El Paso Electric Company',
 'ERCO': 'Electric Reliability Council of TX',
 'FMPP': 'Florida Municipal Power Pool',
 'FPC': 'Duke Energy Florida',
 'FPL': 'Florida Power & Light',
 'GCPD': 'PUD of Grant County, WA',
 'GRIF': 'Griffith Energy',
 'GRIS': 'Gridforce South',
 'GRMA': 'Gila River Power',
 'GVL': 'Gainesville Regional Utilities',
 'GWA': 'NaturEner Power Watch',
 'HECO': 'Hawaiian Electric',
 'HGMA': 'New Harquahala Generating Co.',
 'HST': 'City of Homestead',
 'IID': 'Imperial Irrigation District',
 'IPCO': 'Idaho Power Company',
 'ISNE': 'ISO New England',
 'JEA': 'Jacksonville Electric (JEA)',
 'LDWP': 'LA Dept of Water & Power',
 'LGEE': 'Louisville Gas & Electric',
 'MISO': 'Midcontinent ISO',
 'NBSO': 'New Brunswick System Operator',
 'NEVP': 'Nevada Power Company',
 'NSB': 'New Smyrna Beach',
 'NWMT': 'NorthWestern Energy',
 'NYIS': 'New York ISO',
 'OVEC': 'Ohio Valley Electric Corporation',
 'PACE': 'PacifiCorp - East',
 'PACW': 'PacifiCorp - West',
 'PGE': 'Portland General Electric',
 'PJM': 'PJM Interconnection',
 'PNM': 'PSC of New Mexico',
 'PSCO': 'PSC of Colorado',
 'PSEI': 'Puget Sound Energy',
 'SC': 'South Carolina PSA',
 'SCEG': 'South Carolina Electric & Gas',
 'SCL': 'Seattle City Light',
 'SEC': 'Seminole Electric Cooperative',
 'SEPA': 'Southeastern Power Admin.',
 'SOCO': 'Southern Company',
 'SPA': 'Southwestern Power Admin.',
 'SRP': 'Salt River Project',
 'SWPP': 'Southwest Power Pool',
 'TAL': 'City of Tallahassee',
 'TEC': 'Tampa Electric Company',
 'TEPC': 'Tucson Electric Power',
 'TIDC': 'Turlock Irrigation District',
 'TPWR': 'City of Tacoma',
 'TVA': 'Tennessee Valley Authority',
 'WACM': 'WAPA Rocky Mountain',
 'WALC': 'WAPA Desert Southwest',
 'WAUW': 'WAPA Upper Great Plains West',
 'WWA': 'NaturEner Wind Watch',
 'YAD': 'Alcoa Power Yadkin Division'}

In [ ]:
charge_loss = 0.10

to_plot = results[
    (results["charging_loss"] == charge_loss)
    & (results["gasoline_type"] == "e10_gasoline")
].copy()

to_plot["ev_pct_diff_gas"] = to_plot["ev_pct_diff_gas"] * 100

to_plot = (
    to_plot.groupby(["ba_code", "charging_start", "vehicle_class"]).mean().reset_index()
)

to_plot = to_plot.replace({'ba_code':ba_name_dict})

px.bar(
    to_plot,
    x="ba_code",
    y="ev_pct_diff_gas",
    facet_col="vehicle_class",
    labels={"ba_code": "Grid Region", "ev_pct_diff_gas": "% Difference", "charging_start":"Charge Start Time"},
    facet_col_wrap=1,
    color="charging_start",
    category_orders={"charging_start":["0:00","9:00","12:00","18:00"]},
    color_discrete_map={"0:00":"navy","9:00":"orange","12:00":"gold","18:00":"purple"},
    barmode="group",
    title=f"Average CO2 footprint of an EV compared to an ICE vehicle per mile driven",
    height=800,
).update_yaxes(
    zeroline=True, zerolinecolor="black", zerolinewidth=1, dtick=25, range=[-100, 25]
).for_each_annotation(
    lambda a: a.update(text=a.text.split("=")[-1])
).update_xaxes(
    showgrid=True, ticks="outside", tickson="boundaries", tickangle=45,
).add_hline(y=0, line_color="black", line_width=1)
